Notebook for testing Emergent Abilities

In [2]:
import torch
from torch.nn.utils import prune

from tqdm import tqdm

from transformers import AutoTokenizer, OPTForCausalLM, pipeline
from datasets import load_dataset

from calculate_mask import calculate_mask
from inverse_hessian import calc_inverse_hessian
from input_prehooks import put_input_hooks
from testing_module import calculate_perp

/Users/phillipguo/opt/anaconda3/envs/pyt/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
#DEVICE
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# model_name = "facebook/opt-125m"
# model_name = "facebook/opt-1.3b"
model_name = "facebook/opt-2.7b"

#Load dataset
dataset = load_dataset('c4', 'en', streaming=True)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left', max_new_tokens=30, max_length=100)

# Load model with pre-trained head
# model = OPTForCausalLM.from_pretrained(model_name, output_attentions=True, output_hidden_states=True).to(device=device) # type: ignore

# Load generator
generator = pipeline('text-generation', model=model_name, tokenizer=tokenizer)

Downloading: 100%|██████████| 691/691 [00:00<00:00, 437kB/s]
Downloading: 100%|██████████| 899k/899k [00:00<00:00, 7.28MB/s]
Downloading: 100%|██████████| 456k/456k [00:00<00:00, 4.16MB/s]
Downloading: 100%|██████████| 441/441 [00:00<00:00, 318kB/s]
Downloading: 100%|██████████| 5.30G/5.30G [01:55<00:00, 45.9MB/s]


In [18]:
generator('Hello, my name is', temperature=0.7)

[{'generated_text': 'Hello, my name is John. I am a professional photographer and I am looking for a photographer to'}]

In [48]:
generator('5 + 5 = 10, 13 + 16 = 29, 10 + 15 =', temperature=0.7)

[{'generated_text': '5 + 5 = 10, 13 + 16 = 29, 10 + 15 = 25, 10 +'}]

In [20]:
# digit addition few shot generator
import random
def generate_addition_few_shot(num_dig, num_examples):
    few_shot_str = ""
    for ex in range(num_examples):
        max_num = 10**num_dig-1
        num1 = random.randint(1, max_num)
        num2 = random.randint(1, max_num)
        sum = num1+num2
        few_shot_str += f'{num1} + {num2} = {sum}, '
    return few_shot_str

In [44]:
inp_str = generate_addition_few_shot(2, 10) + " 11 + 25 ="
generator(inp_str, temperature=0.7)

Input length of input_ids is 66, but `max_length` is set to 21. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


[{'generated_text': '86 + 88 = 174, 96 + 61 = 157, 72 + 52 = 124, 21 + 45 = 66, 44 + 81 = 125, 51 + 49 = 100, 23 + 64 = 87, 63 + 36 = 99, 56 + 24 = 80, 89 + 31 = 120,  11 + 25 = 34'}]